In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from datetime import datetime

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv', sep = ',')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv', sep = ',')
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [ ]:
test_date = test['date_time']
#test_date

In [ ]:
train.head()

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], format='%Y-%m-%d %H:%M:%S')
test['date_time'] = pd.to_datetime(test['date_time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
train['month'] = train['date_time'].dt.month
train['day'] = train['date_time'].dt.day
train['hour'] = train['date_time'].dt.hour
train['weekday'] = train['date_time'].dt.dayofweek
#train['minutes'] = train['date_time'].dt.minute 

In [ ]:
test['month'] = test['date_time'].dt.month
test['day'] = test['date_time'].dt.day
test['hour'] = test['date_time'].dt.hour
test['weekday'] = test['date_time'].dt.dayofweek  


In [ ]:
train.tail()

In [ ]:
train = train.drop(['date_time'], axis=1)
test = test.drop(['date_time'], axis=1)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.columns

In [ ]:
tar = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
#lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
X = train.drop(tar, axis = 1)
y = train[tar]

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
sc = StandardScaler()
mi = MinMaxScaler()
#X_train = mi.fit(X_train)
#X_test = mi.transform(X_test)

In [ ]:
#hi

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import xgboost as xg
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=None, min_samples_split=2,random_state=43)
rf = RandomForestRegressor(n_estimators=700, criterion='mse', max_depth=7, random_state = 41)
gbr = GradientBoostingRegressor(n_estimators=700,learning_rate=0.03,subsample=.7,max_depth=7,random_state=43)
xgb_r = xg.XGBRegressor(n_estimators=700, max_depth=7, eta=0.05, subsample=0.7, colsample_bytree=0.8, random_state = 41, booster='gbtree')
mor = MultiOutputRegressor(estimator = gbr)
mor1 = MultiOutputRegressor(estimator = xgb_r)
mor2 = MultiOutputRegressor(estimator = dt)

mor3 = MultiOutputRegressor(estimator = dt)
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

In [ ]:
model = [mor,mor2]
for model in model:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    RMSLE = np.sqrt(mean_squared_log_error(y_test, y_pred))
    print(model,'\n', 'RMSLE:', RMSLE, '\n')

In [ ]:
#from sklearn.model_selection import GridSearchCV
#gsc = GridSearchCV(
#            estimator=xg.XGBRegressor(),
#            param_grid={"learning_rate": (0.03, 0.1),
#                        "max_depth": [ 3, 4, 5, 6],
#                        "min_child_weight": [ 1, 3, 5, 7],
#                        "gamma":[ 0.05, 0.1, 0.2],
#                        "colsample_bytree":[ 0.3, 0.4,0.7],},
#            cv=3, scoring='neg_mean_squared_log_error', verbose=0, n_jobs=-1)

#grid_result = MultiOutputRegressor(gsc).fit(X_train, y_train)

#self.best_params = grid_result.estimators_[0].best_params_

In [ ]:
y_hat = mor.predict(test)


In [ ]:
result = pd.DataFrame(y_hat, columns=['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
result.head()


In [ ]:
result1 = pd.concat([test_date,result], axis=1)
result1.head()

In [ ]:
result1.to_csv('submission.csv',index =False)